# Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yt_dlp
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import re
import glob
import cv2

import variables as var

# Load data

In [2]:
df = pd.read_csv('data/EnTube.csv')
df

,video_id,category,channel_id,engagement_rate_label,q_score,video_link
0,nehrVdADdH0,Film & Animation,UC0jDoh3tVXCaqJ6oTve8ebA,2,0.970364,https://www.youtube.com/watch?v=nehrVdADdH0
1,K66wOEaBwK4,Comedy,UC0jDoh3tVXCaqJ6oTve8ebA,2,0.950070,https://www.youtube.com/watch?v=K66wOEaBwK4
2,D00vn3X7oI8,Entertainment,UC0jDoh3tVXCaqJ6oTve8ebA,2,0.896920,https://www.youtube.com/watch?v=D00vn3X7oI8
3,G22G1k3G-kM,Entertainment,UC0jDoh3tVXCaqJ6oTve8ebA,2,0.930209,https://www.youtube.com/watch?v=G22G1k3G-kM
4,G5EG7ymPErw,Entertainment,UC0jDoh3tVXCaqJ6oTve8ebA,2,0.918029,https://www.youtube.com/watch?v=G5EG7ymPErw
...,...,...,...,...,...,...
23733,a7p51yDbnZM,People & Blogs,UClUMlQD_8cCbof7HxZaXaoQ,0,0.714286,https://www.youtube.com/watch?v=a7p51yDbnZM
23734,DDvnlqUxy1Q,People & Blogs,UClUMlQD_8cCbof7HxZaXaoQ,2,0.944444,https://www.youtube.com/watch?v=DDvnlqUxy1Q
23735,GhxY-7QwD_c,People & Blogs,UClUMlQD_8cCbof7HxZaXaoQ,1,0.866962,https://www.youtube.com/watch?v=GhxY-7QwD_c
23736,MwqkdX9lyuk,People & Blogs,UClUMlQD_8cCbof7HxZaXaoQ,1,0.846154,https://www.youtube.com/watch?v=MwqkdX9lyuk


In [3]:
def find_matching_ids(directory):
    mp4_ids = set()

    for file in os.listdir(directory):
        if file.endswith(".mp4"):
            mp4_ids.add(file[:-4])  # Remove .npz

    return mp4_ids

existed_ids = find_matching_ids(var.DOWNLOADED_VIDEO_PATH)


df = df[df['video_id'].isin(existed_ids)]
df.reset_index(drop=True, inplace=True)
df

,video_id,category,channel_id,engagement_rate_label,q_score,video_link
0,nehrVdADdH0,Film & Animation,UC0jDoh3tVXCaqJ6oTve8ebA,2,0.970364,https://www.youtube.com/watch?v=nehrVdADdH0
1,gHWnW43KROA,Film & Animation,UC0jDoh3tVXCaqJ6oTve8ebA,2,0.926279,https://www.youtube.com/watch?v=gHWnW43KROA
2,dOw0uwkYo3k,Film & Animation,UC0jDoh3tVXCaqJ6oTve8ebA,1,0.865395,https://www.youtube.com/watch?v=dOw0uwkYo3k
3,zjyxjZEJ7Xk,Film & Animation,UC0jDoh3tVXCaqJ6oTve8ebA,2,0.922742,https://www.youtube.com/watch?v=zjyxjZEJ7Xk
4,ueEXHkHYorI,Film & Animation,UC0jDoh3tVXCaqJ6oTve8ebA,2,0.973917,https://www.youtube.com/watch?v=ueEXHkHYorI
...,...,...,...,...,...,...
1454,czTjn-d2IvU,Travel & Events,UClUMlQD_8cCbof7HxZaXaoQ,1,0.800000,https://www.youtube.com/watch?v=czTjn-d2IvU
1455,ectVjZl0LaM,People & Blogs,UClUMlQD_8cCbof7HxZaXaoQ,0,0.600000,https://www.youtube.com/watch?v=ectVjZl0LaM
1456,028H04_nG4M,People & Blogs,UClUMlQD_8cCbof7HxZaXaoQ,2,0.921630,https://www.youtube.com/watch?v=028H04_nG4M
1457,015DqDkdSmU,People & Blogs,UClUMlQD_8cCbof7HxZaXaoQ,2,0.920635,https://www.youtube.com/watch?v=015DqDkdSmU


In [4]:
SAMPLE_ID = 'Ge2WpE4UUiY'
SAMPLE_VIDEO_PATH = f'{var.DOWNLOADED_VIDEO_PATH}/{SAMPLE_ID}.mp4'

In [5]:
def read_srt_file(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        subtitles = f.read()
    return subtitles

# Example usage
srt_file_path = f"{var.DOWNLOADED_VIDEO_PATH}/{SAMPLE_ID}.vi.vtt"  # Replace with actual path
print(read_srt_file(srt_file_path))

WEBVTT
Kind: captions
Language: vi

00:00:00.050 --> 00:00:01.230
Con thưa Cô,

00:00:01.610 --> 00:00:04.300
các bạn sinh năm 2000
 vừa trải qua một kỳ thi đại học.

00:00:04.390 --> 00:00:06.270
Và rất nhiều bạn ngoài việc lựa chọn

00:00:06.290 --> 00:00:08.690
 một trường đại học ở trong nước

00:00:08.780 --> 00:00:10.500
thì rất nhiều bạn 
cũng có định hướng

00:00:10.620 --> 00:00:12.640
và có những cái ước mơ đi du học.

00:00:12.900 --> 00:00:15.380
Và câu hỏi của một bạn đang rất đắn đo

00:00:15.390 --> 00:00:17.540
về con đường bạn ấy chuẩn bị đi tới đây,

00:00:17.540 --> 00:00:19.650
đó là con đường xuất ngoại, đi du học.

00:00:19.820 --> 00:00:21.380
Con xin phép được đọc câu hỏi.

00:00:21.560 --> 00:00:22.530
“Con thưa Cô!”

00:00:22.680 --> 00:00:24.200
“Năm nay, con 19 tuổi,”

00:00:24.270 --> 00:00:26.620
“sắp tới con sẽ đi du học ở nước ngoài.”

00:00:26.700 --> 00:00:30.180
“Con biết tâm lý khi xa gia đình, 
người thân và người quen,”

00:00:30.270 --> 00:00:32.4

In [6]:
import ffmpeg

def get_video_info(video_path):
    """ Extracts resolution, fps, and audio sample rate from a video file. """
    probe = ffmpeg.probe(video_path)

    # Extract video stream info
    video_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
    audio_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'audio'), None)

    if video_stream:
        width = video_stream['width']
        height = video_stream['height']
        fps = eval(video_stream['r_frame_rate'])  # Convert fraction to float
    else:
        width = height = fps = None

    if audio_stream:
        sample_rate = int(audio_stream['sample_rate'])
    else:
        sample_rate = None

    return {
        "resolution": f"{width}x{height}" if width and height else None,
        "fps": fps,
        "audio_sample_rate": sample_rate
    }

# Example usage
video_file = os.path.join(var.DOWNLOADED_VIDEO_PATH, f"{SAMPLE_ID}.mp4")
info = get_video_info(video_file)
print(info)

{'resolution': '640x360', 'fps': 25.0, 'audio_sample_rate': 48000}


In [7]:

import cv2
import numpy as np
import ffmpeg
from pydub import AudioSegment
import io

def video_audio_to_array(video_path, 
                         target_height=var.TARGET_FRAME_SIZE[0], target_width=var.TARGET_FRAME_SIZE[1], target_fps=var.VIDEO_FPS, 
                         target_sample_rate=var.AUDIO_FPS):
    """Extracts video frames and audio from an mp4 file, ensuring fixed FPS, resizing frames with padding,
       and downsampling audio to a lower sample rate while preserving speech.

    Args:
        video_path (str): Path to the video file.
        target_height (int): Target video frame height.
        target_width (int): Target video frame width.
        target_fps (int): Target frame rate.
        target_sample_rate (int): Target audio sample rate (default 16 kHz for speech).

    Returns:
        video_array (np.ndarray): (num_frames, target_height, target_width, 3), dtype=np.uint8.
        audio_array (np.ndarray): (num_samples,) or (num_samples, 2) for stereo, dtype=np.int16.
        sample_rate (int): Audio sample rate in Hz.
    """
    # ---- Extract Video Frames with Fixed FPS ----
    cap = cv2.VideoCapture(video_path)
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = original_fps / target_fps  # Frame selection interval

    frames = []
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % round(frame_interval) == 0:  # Select frames to match target FPS
            h, w, _ = frame.shape

            # Resize with padding to maintain aspect ratio
            scale = min(target_width / w, target_height / h)
            new_w, new_h = int(w * scale), int(h * scale)
            resized_frame = cv2.resize(frame, (new_w, new_h))

            resized_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)

            # Pad to fixed size
            padded_frame = np.zeros((target_height, target_width, 3), dtype=np.uint8)
            pad_top = (target_height - new_h) // 2
            pad_left = (target_width - new_w) // 2
            padded_frame[pad_top:pad_top+new_h, pad_left:pad_left+new_w] = resized_frame

            frames.append(padded_frame)  

        frame_idx += 1

    cap.release()
    video_array = np.array(frames, dtype=np.uint8)

    # ---- Extract and Downsample Audio ----
    out, _ = (
        ffmpeg.input(video_path)
        .output("pipe:", format="wav", 
                ar=target_sample_rate, # Resample to target sample rate
                ac=1)               # Mono audio
        .run(capture_stdout=True, capture_stderr=True)
    )

    audio = AudioSegment.from_file(io.BytesIO(out), format="wav")
    samples = np.array(audio.get_array_of_samples(), dtype=np.int16)

    return video_array, samples # sh

# # Example usage
# video_array, audio_array = video_audio_to_array(SAMPLE_VIDEO_PATH)

# print(f"Original Video path: {SAMPLE_VIDEO_PATH}") 
# print(f"Video shape: {video_array.shape}")  # (num_frames, target_height, target_width, 3)
# print(f"Audio shape: {audio_array.shape}")  # (num_samples,)


# plt.imshow(video_array[80])
# plt.show()

# Note: not works for WSL
# import sounddevice as sd
# audio_samples = audio_samples / np.max(np.abs(audio_samples))
# print("Playing audio...")
# sd.play(audio_samples, samplerate=AUDIO_FPS)
# sd.wait()

In [8]:
import numpy as np
import wave

def save_video_audio(video_array, audio_array, sample_rate, video_file, audio_file):
    np.savez_compressed(video_file, video=video_array)
    
    # Save audio as WAV (16-bit PCM, uncompressed for fast loading)
    with wave.open(audio_file, 'w') as wf:
        wf.setnchannels(1)  # Mono
        wf.setsampwidth(2)  # 16-bit PCM (2 bytes per sample)
        wf.setframerate(sample_rate)
        wf.writeframes(audio_array.tobytes())

# Example usage
# save_video_audio(video_array, audio_array, var.AUDIO_FPS, video_file="temp/video.npz", audio_file="temp/audio.wav")

# Split to scenes

In [16]:
from scenedetect import detect, AdaptiveDetector, ContentDetector #  AdaptiveDetector, split_video_ffmpeg

def extract_scenes(video_path):
    """Detect scene changes and return clip start/end times."""
    scene_list = detect(video_path, ContentDetector())
    return scene_list  # List of (start_time, end_time)

In [14]:
samplee = df['video_id'].sample(5)

In [17]:
# Example: Process all videos in the dataset
# df = pd.read_csv("dataset.csv")  # Your video metadata
scene_data = []

for id in samplee:
    print(id)
    scenes = extract_scenes(os.path.join(var.DOWNLOADED_VIDEO_PATH, id + ".mp4"))
    for start, end in scenes:
        scene_data.append({"id": id, "start": start, "end": end})

# Convert scene data to a DataFrame
scene_df = pd.DataFrame(scene_data)
scene_df.to_csv("scene_data.csv", index=False)  # Save for later use

tg2Mc530xr4
OzZdvbhm8jg
owv1VPO41JA
CDGsT2Ukh_Q
NgMmVZ_QH8s


# Create dataset

In [ ]:
import torch
from torch.utils.data import Dataset
from pytorchvideo.data.encoded_video import EncodedVideo

class VideoDataset(Dataset):
    def __init__(self, ids, labels, transform=None):
        self.ids = ids
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        video_path = os.path.join(var.DOWNLOADED_VIDEO_PATH, f"{self.ids[idx]}.mp4")
        label = self.labels[idx]
        
        video = EncodedVideo.from_path(video_path)  # Load full video
        
        # Sample a random start time within the video duration
        duration = video.duration
        start_time = torch.randint(0, max(1, int(duration - self.clip_duration)), (1,)).item()
        end_time = start_time + self.clip_duration

        # Extract clip
        clip = video.get_clip(start_sec=start_time, end_sec=end_time)
        video_tensor = clip["video"]  # Shape: (C, T, H, W)

        if self.transform:
            video_tensor = self.transform(video_tensor)

        return video_tensor, torch.tensor(label)


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from pytorchvideo.models.hub import slowfast_r50  # Load SlowFast model

# Load the pretrained SlowFast model
model = slowfast_r50(pretrained=False)
# Set to GPU or CPU
model = model.eval()
# model = model.to(device)

In [ ]:
# Modify the classifier head to output 3 classes
num_classes = 3
model.blocks[-1].proj = nn.Linear(model.blocks[-1].proj.in_features, num_classes)

print(model)

In [ ]:
import torch.optim as optim

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

# Move model to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for batch in train_loader:
        video_inputs = batch["video"].to(device)  # Video tensor (B, C, T, H, W)
        labels = batch["label"].to(device)  # Class labels
        
        optimizer.zero_grad()
        outputs = model(video_inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")
